In [1]:
import rdflib
from rdflib.graph import Graph, URIRef
from SPARQLWrapper import SPARQLWrapper, XML


In [5]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

construct_query= """
PREFIX ma: <http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>

CONSTRUCT {
  ?book rdf:type ma:Book .
  ?book ma:title ?name .
  ?book ma:written_by ?author .
  ?book ma:publisher_of ?publisher .
  ?publisher rdf:type ma:publisher .
  ?book ma:literaryGenre_of ?genre .
  ?genre rdf:type ma:literaryGenre .
  ?book ma:releaseDate ?releaseDate .
  ?book ma:Country ?Country .
  ?book rdf:type ma:character .
  ?character rdf:type ma:character .
}
WHERE {
  ?book wdt:P31 wd:Q571 .
  ?book wdt:P1476 ?name .
  FILTER (LANG(?name) = 'en') .

  OPTIONAL { ?book wdt:P50 ?author . }
  OPTIONAL { ?book wdt:P136 ?genre . }
  OPTIONAL { ?book wdt:P123 ?publisher . }
  ?book wdt:P577 ?releaseDate .
  OPTIONAL { ?book wdt:P495 ?Country . }
}"""

sparql.setQuery(construct_query)
sparql.setReturnFormat(XML)

graph_id = URIRef("http://www.semanticweb.org/store/book")
g = Graph(identifier=graph_id)

print("I might take some time, bear with me...")

results = sparql.query().convert()

g += results

g.serialize("book_wiki.owl", format="xml")

print("...All done!")
print("")

I might take some time, bear with me...
...All done!

